Dieses Notebook ist Teil des Grundprojekts "enchmarking von Retrievalmethoden in RAG-Systemen: Ein Vergleich zwischen Vektor- und Graph-basiertem Retrieval".
Ziel ist die Vorbereitung des Datenmaterials für die Experimente mit Vektor- und Graph-Retrieval. Grundlage bilden 15 FAQ-Dokumente mit Fragen und Antworten.
Vgl. Gao et al. (2023) für RAG-Grundlagen und Sarmah et al. (2023) für Graph-Retrieval.

In [1]:
import json
import pandas as pd
from pathlib import Path


In [2]:
faq_documents = [
    {
        "id": "doc_001",
        "question": "Was ist Retrieval-Augmented Generation (RAG)?",
        "answer": "RAG ist eine Technik, die Large Language Models mit externen Wissensquellen verbindet. Das Modell kann relevante Informationen aus einer Wissensbasis abrufen und diese in seine Antworten einbeziehen. Dies reduziert Halluzinationen und ermöglicht aktuellere Informationen.",
        "category": "RAG Basics",
        "keywords": ["RAG", "retrieval", "language model", "knowledge base"]
    },
    {
        "id": "doc_002",
        "question": "Wie funktioniert Vektorsuche in RAG-Systemen?",
        "answer": "Bei der Vektorsuche werden Dokumente und Anfragen in hochdimensionale Vektoren (Embeddings) umgewandelt. Die Ähnlichkeit wird durch Cosinus-Ähnlichkeit oder andere Metriken gemessen. FAISS oder ähnliche Bibliotheken ermöglichen schnelle Suche in großen Vektorräumen.",
        "category": "Vector Retrieval",
        "keywords": ["vector search", "embeddings", "FAISS", "similarity"]
    },
    {
        "id": "doc_003",
        "question": "Was sind die Vorteile von Graph-basiertem Retrieval?",
        "answer": "Graph-basiertes Retrieval kann Beziehungen zwischen Entitäten erfassen und komplexe, multi-hop Reasoning ermöglichen. Es ist besonders stark bei strukturierten Daten und kann Kontext durch Graphtraversierung liefern. Knowledge Graphs bieten explizite semantische Beziehungen.",
        "category": "Graph Retrieval",
        "keywords": ["graph retrieval", "knowledge graph", "relationships", "reasoning"]
    },
    {
        "id": "doc_004",
        "question": "Welche Evaluationsmetriken gibt es für RAG-Systeme?",
        "answer": "Häufige Metriken sind BLEU und ROUGE für Textähnlichkeit, sowie Faithfulness und Answer Relevancy. GPT-as-a-Judge wird oft für qualitative Bewertung genutzt. Retrieval-Metriken wie Precision@K und Recall@K messen die Qualität der abgerufenen Dokumente.",
        "category": "Evaluation",
        "keywords": ["BLEU", "ROUGE", "evaluation", "metrics", "GPT-judge"]
    },
    {
        "id": "doc_005",
        "question": "Was ist der Unterschied zwischen Dense und Sparse Retrieval?",
        "answer": "Dense Retrieval nutzt neuronale Embeddings für semantische Ähnlichkeit, während Sparse Retrieval auf exakte Keyword-Matches setzt (wie BM25). Dense Retrieval ist besser für semantische Suche, Sparse Retrieval für exakte Begriff-Suche. Hybrid-Ansätze kombinieren beide.",
        "category": "Retrieval Methods",
        "keywords": ["dense retrieval", "sparse retrieval", "BM25", "embeddings"]
    },
    {
        "id": "doc_006",
        "question": "Wie wählt man die richtige Chunk-Größe für RAG?",
        "answer": "Die Chunk-Größe hängt vom Use Case ab. Kleine Chunks (100-200 Tokens) bieten präzise Retrieval, große Chunks (500-1000 Tokens) mehr Kontext. Overlap zwischen Chunks verhindert Informationsverlust. Experimentierung mit verschiedenen Größen ist empfohlen.",
        "category": "Data Processing",
        "keywords": ["chunking", "token size", "overlap", "preprocessing"]
    },
    {
        "id": "doc_007",
        "question": "Was sind häufige Probleme bei RAG-Implementierungen?",
        "answer": "Häufige Probleme sind: irrelevante Retrieval-Ergebnisse, Halluzinationen trotz RAG, hohe Latenz, Context-Window-Überschreitungen und schlechte Chunk-Qualität. Lösungen umfassen bessere Embeddings, Re-ranking, und optimierte Chunk-Strategien.",
        "category": "Troubleshooting",
        "keywords": ["problems", "hallucination", "latency", "context window"]
    },
    {
        "id": "doc_008",
        "question": "Welche Rolle spielt LangChain in RAG-Systemen?",
        "answer": "LangChain bietet einen einheitlichen Framework für RAG-Implementierungen. Es abstrahiert Retriever, Vector Stores und LLM-Integrationen. Chain-Komponenten ermöglichen modulare Pipelines. LangChain vereinfacht die Integration verschiedener Tools und Datenquellen.",
        "category": "Tools",
        "keywords": ["LangChain", "framework", "chains", "integration"]
    },
    {
        "id": "doc_009",
        "question": "Wie funktioniert Re-ranking in RAG-Pipelines?",
        "answer": "Re-ranking verbessert die initial abgerufenen Dokumente durch ein zweites Modell. Cross-encoder oder spezialisierte Re-ranking-Modelle bewerten Relevanz präziser als die erste Retrieval-Stufe. Dies kann die Antwortqualität signifikant verbessern.",
        "category": "Advanced Techniques",
        "keywords": ["re-ranking", "cross-encoder", "two-stage retrieval"]
    },
    {
        "id": "doc_010",
        "question": "Was ist Hybrid Retrieval und wann sollte man es nutzen?",
        "answer": "Hybrid Retrieval kombiniert verschiedene Retrieval-Methoden, typischerweise Dense und Sparse. Es nutzt die Stärken beider Ansätze: semantische Suche und exakte Keyword-Matches. Besonders effektiv bei diversen Query-Typen und großen Wissensbasen.",
        "category": "Advanced Techniques",
        "keywords": ["hybrid retrieval", "combination", "dense", "sparse"]
    },
    {
        "id": "doc_011",
        "question": "Welche Vector Databases sind für RAG geeignet?",
        "answer": "Beliebte Optionen sind Pinecone, Weaviate, Qdrant und Chroma für managed Services. FAISS eignet sich für lokale Implementierungen. Die Wahl hängt von Skalierung, Latenz-Anforderungen und Budget ab. PostgreSQL mit pgvector ist eine kostengünstige Alternative.",
        "category": "Infrastructure",
        "keywords": ["vector database", "Pinecone", "FAISS", "Chroma"]
    },
    {
        "id": "doc_012",
        "question": "Wie optimiert man die Embedding-Qualität für RAG?",
        "answer": "Fine-tuning von Embedding-Modellen auf domänenspezifische Daten verbessert Retrieval-Qualität. Auch die Wahl des Base-Models (OpenAI, Sentence-Transformers) ist wichtig. Preprocessing, Normalisierung und geeignete Similarity-Metriken spielen eine Rolle.",
        "category": "Optimization",
        "keywords": ["embeddings", "fine-tuning", "domain-specific", "similarity"]
    },
    {
        "id": "doc_013",
        "question": "Was sind die Kosten-Überlegungen bei RAG-Systemen?",
        "answer": "Hauptkostenfaktoren sind LLM-API-Calls, Vector-Database-Hosting und Embedding-Generierung. Caching von Embeddings und Antworten reduziert Kosten. Lokale Modelle können API-Kosten eliminieren, benötigen aber eigene Infrastruktur.",
        "category": "Business",
        "keywords": ["costs", "pricing", "API", "caching", "local models"]
    },
    {
        "id": "doc_014",
        "question": "Wie implementiert man RAG mit Neo4j?",
        "answer": "Neo4j ermöglicht Graph-basiertes RAG durch Cypher-Queries. Entitäten und Beziehungen werden extrahiert und als Graph gespeichert. Retrieval erfolgt durch Graphtraversierung statt Vektorsuche. Dies ermöglicht komplexere, beziehungsbasierte Abfragen.",
        "category": "Graph Implementation",
        "keywords": ["Neo4j", "Cypher", "graph traversal", "entities"]
    },
    {
        "id": "doc_015",
        "question": "Welche Zukunftstrends gibt es bei RAG-Technologien?",
        "answer": "Trends umfassen Agentic RAG mit Tool-Usage, Multi-modal RAG für Bilder/Videos, und Adaptive Retrieval basierend auf Query-Komplexität. Knowledge Graph-Integration wird wichtiger, ebenso Real-time Updates und personalisierte Retrieval-Strategien.",
        "category": "Future Trends",
        "keywords": ["agentic RAG", "multi-modal", "adaptive retrieval", "trends"]
    }
]

print(f"FAQ-Korpus erstellt: {len(faq_documents)} Dokumente")

FAQ-Korpus erstellt: 15 Dokumente


## Testfragen erstellen

12 Fragen verschiedener Schwierigkeitsgrade für das Benchmarking.

In [3]:
# Testfragen für das Benchmarking
test_questions = [
    {
        "id": "q001",
        "question": "Erkläre mir RAG in einfachen Worten",
        "expected_topics": ["RAG", "retrieval", "language model"],
        "difficulty": "easy",
        "category": "basic"
    },
    {
        "id": "q002",
        "question": "Was sind die Unterschiede zwischen Vektor- und Graph-Retrieval?",
        "expected_topics": ["vector search", "graph retrieval", "comparison"],
        "difficulty": "medium",
        "category": "comparison"
    },
    {
        "id": "q003",
        "question": "Welche Evaluationsmetriken sollte ich für mein RAG-System verwenden?",
        "expected_topics": ["BLEU", "ROUGE", "evaluation", "metrics"],
        "difficulty": "medium",
        "category": "technical"
    },
    {
        "id": "q004",
        "question": "Wie optimiere ich die Performance meines RAG-Systems?",
        "expected_topics": ["optimization", "chunking", "embeddings", "re-ranking"],
        "difficulty": "hard",
        "category": "optimization"
    },
    {
        "id": "q005",
        "question": "Was kostet der Betrieb eines RAG-Systems?",
        "expected_topics": ["costs", "pricing", "infrastructure"],
        "difficulty": "easy",
        "category": "business"
    },
    {
        "id": "q006",
        "question": "Wie implementiere ich Hybrid Retrieval?",
        "expected_topics": ["hybrid retrieval", "dense", "sparse", "combination"],
        "difficulty": "hard",
        "category": "technical"
    },
    {
        "id": "q007",
        "question": "Welche Vector Database sollte ich wählen?",
        "expected_topics": ["vector database", "FAISS", "Pinecone"],
        "difficulty": "medium",
        "category": "infrastructure"
    },
    {
        "id": "q008",
        "question": "Was sind häufige Probleme bei RAG und wie löse ich sie?",
        "expected_topics": ["problems", "troubleshooting", "hallucination"],
        "difficulty": "medium",
        "category": "troubleshooting"
    },
    {
        "id": "q009",
        "question": "Wie funktioniert RAG mit Knowledge Graphs?",
        "expected_topics": ["Neo4j", "knowledge graph", "Cypher"],
        "difficulty": "hard",
        "category": "graph"
    },
    {
        "id": "q010",
        "question": "Was ist der optimale Chunk-Size für meine Dokumente?",
        "expected_topics": ["chunking", "token size", "preprocessing"],
        "difficulty": "medium",
        "category": "preprocessing"
    },
    {
        "id": "q011",
        "question": "Welche Zukunftstrends gibt es bei RAG?",
        "expected_topics": ["future trends", "agentic RAG", "multi-modal"],
        "difficulty": "easy",
        "category": "trends"
    },
    {
        "id": "q012",
        "question": "Wie verbessere ich meine Embedding-Qualität?",
        "expected_topics": ["embeddings", "fine-tuning", "optimization"],
        "difficulty": "hard",
        "category": "optimization"
    }
]

print(f"Testfragen erstellt: {len(test_questions)} Fragen")
print(f"   - Easy: {len([q for q in test_questions if q['difficulty'] == 'easy'])}")
print(f"   - Medium: {len([q for q in test_questions if q['difficulty'] == 'medium'])}")
print(f"   - Hard: {len([q for q in test_questions if q['difficulty'] == 'hard'])}")

Testfragen erstellt: 12 Fragen
   - Easy: 3
   - Medium: 5
   - Hard: 4


## Daten speichern

In [6]:
with open("../data/faq_korpus.json", "w", encoding="utf-8") as f:
    json.dump(faq_documents, f, ensure_ascii=False, indent=2)

print("FAQ-Korpus gespeichert: ../data/faq_korpus.json")

FAQ-Korpus gespeichert: ../data/faq_corpus.json


In [7]:
# Testfragen als CSV speichern (für bessere Übersicht)
questions_df = pd.DataFrame(test_questions)
questions_df.to_csv('../data/fragenliste.csv', index=False, encoding='utf-8')

print("Testfragen gespeichert: ../data/fragenliste.csv")
print("\nÜbersicht der Testfragen:")
display(questions_df[['id', 'question', 'difficulty', 'category']])

Testfragen gespeichert: ../data/fragenliste.csv

Übersicht der Testfragen:


,id,question,difficulty,category
0,q001,Erkläre mir RAG in einfachen Worten,easy,basic
1,q002,Was sind die Unterschiede zwischen Vektor- und...,medium,comparison
2,q003,Welche Evaluationsmetriken sollte ich für mein...,medium,technical
3,q004,Wie optimiere ich die Performance meines RAG-S...,hard,optimization
4,q005,Was kostet der Betrieb eines RAG-Systems?,easy,business
5,q006,Wie implementiere ich Hybrid Retrieval?,hard,technical
6,q007,Welche Vector Database sollte ich wählen?,medium,infrastructure
7,q008,Was sind häufige Probleme bei RAG und wie löse...,medium,troubleshooting
8,q009,Wie funktioniert RAG mit Knowledge Graphs?,hard,graph
9,q010,Was ist der optimale Chunk-Size für meine Doku...,medium,preprocessing


## Überblick der erstellten Daten

In [8]:
# Übersicht FAQ-Korpus
print("=== FAQ-KORPUS ÜBERSICHT ===")
print(f"Anzahl Dokumente: {len(faq_documents)}")

# Kategorien anzeigen
categories = [doc['category'] for doc in faq_documents]
category_counts = pd.Series(categories).value_counts()
print("\nKategorien:")
for cat, count in category_counts.items():
    print(f"  - {cat}: {count} Dokumente")

# Beispiel-Dokument
print("\n=== BEISPIEL-DOKUMENT ===")
example = faq_documents[0]
print(f"ID: {example['id']}")
print(f"Frage: {example['question']}")
print(f"Antwort: {example['answer'][:100]}...")
print(f"Keywords: {example['keywords']}")

=== FAQ-KORPUS ÜBERSICHT ===
Anzahl Dokumente: 15

Kategorien:
  - Advanced Techniques: 2 Dokumente
  - RAG Basics: 1 Dokumente
  - Graph Retrieval: 1 Dokumente
  - Evaluation: 1 Dokumente
  - Retrieval Methods: 1 Dokumente
  - Vector Retrieval: 1 Dokumente
  - Data Processing: 1 Dokumente
  - Troubleshooting: 1 Dokumente
  - Tools: 1 Dokumente
  - Infrastructure: 1 Dokumente
  - Optimization: 1 Dokumente
  - Business: 1 Dokumente
  - Graph Implementation: 1 Dokumente
  - Future Trends: 1 Dokumente

=== BEISPIEL-DOKUMENT ===
ID: doc_001
Frage: Was ist Retrieval-Augmented Generation (RAG)?
Antwort: RAG ist eine Technik, die Large Language Models mit externen Wissensquellen verbindet. Das Modell ka...
Keywords: ['RAG', 'retrieval', 'language model', 'knowledge base']


In [9]:
print("\n=== SETUP ABGESCHLOSSEN ===")
print("FAQ-Korpus: 15 Dokumente erstellt")
print("Testfragen: 12 Fragen formuliert")
print("Dateien gespeichert in /data/")


=== SETUP ABGESCHLOSSEN ===
FAQ-Korpus: 15 Dokumente erstellt
Testfragen: 12 Fragen formuliert
Dateien gespeichert in /data/
